In [1]:
import numpy as np
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
pv.set_jupyter_backend('ipyvtklink')

In [2]:
cmap = plt.cm.get_cmap("viridis")
pv.global_theme.background = 'white'
pv.global_theme.font.color = 'black'

In [3]:
#greenland data
sim_1 = pd.read_csv("Output/sim_1.csv")
sim_2 = pd.read_csv("Output/sim_3.csv")
bm = pd.read_csv("Output/BM_greenland_sim.csv")
flight_data = pd.read_csv("Data/test_data.csv")

#antarctica data
sim_1_pig = pd.read_csv("Output/sim_1_PIG.csv")
sim_2_pig = pd.read_csv("Output/sim_3_PIG.csv")
bm_pig = pd.read_csv("Output/BM_antarctica_sim.csv")
flight_data_pig = pd.read_csv("Data/PIG_data.csv")


In [4]:
def make_mesh(data, x, y, bed, mesh = True):
    '''
    Makes a pyvista Polydata mesh/plot from given dataset
    
    INPUT
    data = Pandas Dataframe
    x = column name for x values
    y = column name for y values
    bed = column name for elevation
    mesh = delaunay triangulation to create mesh if true, regular plot if false
    
    OUTPUT
    mesh = resulting mesh/plot
    datrange = min and max values from the data. Useful for clim standardization when plotting.
    '''
    plot = pv.PolyData(data[[x,y,bed]].to_numpy())  # Converts to Polydata
    plot.point_data["Elevation"] = data[bed]        # Creates scalar for elevation--colormap support
    if (mesh):
        mesh = plot.delaunay_2d()                   # Used delaunay triangulation to make mesh. 
                                                    # Not recommended for sparse data (conditioning)
    else:
        mesh = plot
        
    datrange = (data[bed].min(), data[bed].max())
    return mesh, datrange

In [5]:
#Greenland data params
mesh_flight, clim = make_mesh(flight_data,"X","Y","Bed", mesh = False)
mesh_1, clim_1 = make_mesh(sim_1,"X","Y","Bed")
mesh_2, clim_2 = make_mesh(sim_2,"X","Y","Bed")
mesh_bm, clim_bm = make_mesh(bm,"x","y","bed")

#Antarctica data params
mesh_flight_pig, clim_pig = make_mesh(flight_data_pig,"x","y","bedrock_altitude (m)", mesh = False)
mesh_1_pig, clim_1 = make_mesh(sim_1_pig,"x","y","bedrock_altitude (m)")
mesh_2_pig, clim_2 = make_mesh(sim_2_pig,"x","y","bedrock_altitude (m)")
mesh_bm_pig, clim_bm = make_mesh(bm_pig,"x","y","bed")

In [6]:
#creating arrows to indicate flow for the greenland/antarctica data
arrow_test = pv.Arrow(start = (-200000,-1720000,10000),
                 direction = (-1, 0,0),
                 scale = 50000)

arrow_pig = pv.Arrow(start = (-1570000,-140000,500),
                 direction = (0,-1,0),
                 scale = 50000)

In [7]:
def pvplot(mesh, clim, title, bounds, add_arrow = False, arrow = 0, ):
    '''
    Creates plotter object based on given mesh.
    
    INPUT
    mesh = PolyData mesh/plot
    clim = tuple containing two values, limits the range of the constructed colorbar
    title = Title of plot. Will appear in top left
    add_arrow = If true, will plot 'arrow'
    arrow = Arrow mesh created separately
    
    OUTPUT
    plotter = plotter object
    '''
    plotter = pv.Plotter(off_screen =True)

    sargs = dict(
        title = "Elevation (m)",
        vertical = False, 
        position_x = 0.4, 
        position_y = 0.9,
        n_labels = 4
    )
    plotter.add_text(title)
    plotter.add_mesh(mesh, cmap=cmap, scalar_bar_args = sargs, clim = clim)
    plotter.show_bounds(show_zlabels=False,
                        show_zaxis=False,
                        xlabel="X (km)",
                        ylabel = "Y (km)", 
                        location = "outer", 
                        font_size = 30,
                        axes_ranges = bounds
                       )
    
    if(add_arrow):
        plotter.add_mesh(arrow,color='red',lighting=True)

    plotter.camera_position = "xz"
    plotter.camera.azimuth = 45
    plotter.camera.elevation = 25
    plotter.camera.zoom(1.3)
    
    return plotter

In [8]:
bound_gn = [-300,-150,-1800,-1650,0,0]
bound_pig = [-1620,-1520,-250, -110, 0, 0]
clim = (-600,600)
clim_pig = (-1700, 400)

In [9]:
gn_flight = pvplot(mesh_flight, clim, "Conditioning Data", bound_gn, add_arrow = True, arrow = arrow_test)
gn_flight.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [10]:
gn_1 = pvplot(mesh_1, clim, "SGS 1", bounds = [-300,-150,-1800,-1600,0,0], add_arrow = True, arrow = arrow_test)
gn_1.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [11]:
gn_2 = pvplot(mesh_2, clim, "SGS 2",bounds = [-300,-150,-1800,-1600,0,0], add_arrow = True, arrow = arrow_test)
gn_2.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [12]:
gn_bm = pvplot(mesh_bm, clim, "Bedmachine",bounds = [-300,-150,-1800,-1600,0,0], add_arrow = True, arrow = arrow_test)
gn_bm.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [19]:
#write to pdf. Change path and file format as needed
gn_flight.save_graphic("Output/graphic/Final/gn_flight.pdf")
gn_1.save_graphic("Output/graphic/Final/gn_sim_1.pdf")
gn_2.save_graphic("Output/graphic/Final/gn_sim_2.pdf")
gn_bm.save_graphic("Output/graphic/Final/gn_sim_bm.pdf")

In [14]:
pig_flight = pvplot(mesh_flight_pig, clim_pig, "Conditioning Data", bound_pig, add_arrow = True, arrow = arrow_pig)
pig_flight.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [15]:
pig_1 = pvplot(mesh_1_pig, clim_pig, "SGS 1", bound_pig, add_arrow = True, arrow = arrow_pig)
pig_1.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [16]:
pig_2 = pvplot(mesh_2_pig, clim_pig, "SGS 2", bound_pig, add_arrow = True, arrow = arrow_pig)
pig_2.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [17]:
pig_bm = pvplot(mesh_bm_pig, clim_pig, "Bedmachine", bound_pig, add_arrow = True, arrow = arrow_pig)
pig_bm.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [18]:
#write to pdf. Change path and file format as needed
pig_flight.save_graphic("Output/graphic/Final/pig_flight.pdf")
pig_1.save_graphic("Output/graphic/Final/pig_sim_1.pdf")
pig_2.save_graphic("Output/graphic/Final/pig_sim_2.pdf")
pig_bm.save_graphic("Output/graphic/Final/pig_sim_bm.pdf")